In [8]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs

In [9]:
# test case list 
# test_case 0 -> Use CT Data to detect cancer (Keras Neural Network)
# test_case 1 -> Use CT Data to detect cancer (KNN)
# test_case 2 -> Use CT Data + Clinic Data to detect death (Keras Neural Network)
test_case = 2

In [10]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()
def kFoldCV(k, X, y, model):
    len_x = X.shape[0]
    ind_step = int(len_x  / k)
    ind = np.array([i * ind_step for i in range(k)])
    accuracy = np.zeros(k)
    precision = np.zeros(k)
    recall = np.zeros(k)
    # first k - 1 fold
    for i in range(k - 1):
        X_train_k = np.vstack((X[:ind[i],:],X[ind[i+1]:,:]))
        X_test_k = X[ind[i]:ind[i + 1]]
        y_train_k = np.hstack((y[:ind[i]],y[ind[i+1]:]))
        y_test_k = y[ind[i]:ind[i + 1]]
        _, accuracy[i],precision[i],recall[i] = model.evaluate(X_test_k, y_test_k)       
    # last fold
    X_train_k = X[:ind[k - 1],:]
    y_train_k = y[:ind[k - 1]]
    X_test_k = X[ind[k - 1]:,:]
    y_test_k = y[ind[k - 1]:]
    _, accuracy[k - 1],precision[k - 1],recall[k - 1] = model.evaluate(X_test_k, y_test_k)
    return accuracy,precision,recall

# Test Case 0 - Keras CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. This test run uses a neural network constructed using Keras

In [4]:
if test_case == 0:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=11, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=700, batch_size=50,verbose=1)
    # evaluate the keras model
    accuracy,precision,recall = kFoldCV(10, X, y, model)
    print("K fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
          .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    cancer_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                cancer_dectected = cancer_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(cancer_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

Epoch 1/700
185/185 [==============================] - 1s 925us/step - loss: 1.3991 - accuracy: 0.7648 - precision: 0.1232 - recall: 0.1299  
Epoch 2/700
185/185 [==============================] - 0s 893us/step - loss: 0.5224 - accuracy: 0.7749 - precision: 0.1642 - recall: 0.1754
Epoch 3/700
185/185 [==============================] - 0s 936us/step - loss: 0.4705 - accuracy: 0.7722 - precision: 0.1403 - recall: 0.1439
Epoch 4/700
185/185 [==============================] - 0s 933us/step - loss: 0.4321 - accuracy: 0.7746 - precision: 0.1458 - recall: 0.1481
Epoch 5/700
185/185 [==============================] - 0s 971us/step - loss: 0.6956 - accuracy: 0.7755 - precision: 0.1601 - recall: 0.1679
Epoch 6/700
185/185 [==============================] - 0s 949us/step - loss: 0.4708 - accuracy: 0.7802 - precision: 0.1557 - recall: 0.1530
Epoch 7/700
185/185 [==============================] - 0s 961us/step - loss: 0.5163 - accuracy: 0.7829 - precision: 0.1620 - recall: 0.1572
Epoch 8/700
185/18

185/185 [==============================] - 0s 938us/step - loss: 0.2324 - accuracy: 0.8202 - precision: 0.2266 - recall: 0.1538
Epoch 60/700
185/185 [==============================] - 0s 951us/step - loss: 0.2294 - accuracy: 0.8182 - precision: 0.2038 - recall: 0.1332
Epoch 61/700
185/185 [==============================] - 0s 948us/step - loss: 0.3189 - accuracy: 0.7990 - precision: 0.1726 - recall: 0.1406
Epoch 62/700
185/185 [==============================] - 0s 944us/step - loss: 0.2182 - accuracy: 0.8254 - precision: 0.2172 - recall: 0.1274
Epoch 63/700
185/185 [==============================] - 0s 915us/step - loss: 0.2144 - accuracy: 0.8125 - precision: 0.1991 - recall: 0.1423
Epoch 64/700
185/185 [==============================] - 0s 928us/step - loss: 0.1700 - accuracy: 0.8298 - precision: 0.2261 - recall: 0.1232
Epoch 65/700
185/185 [==============================] - 0s 992us/step - loss: 0.2741 - accuracy: 0.8152 - precision: 0.2158 - recall: 0.1555
Epoch 66/700
185/185 [====

185/185 [==============================] - 0s 939us/step - loss: 0.1133 - accuracy: 0.8594 - precision: 0.3713 - recall: 0.1050
Epoch 118/700
185/185 [==============================] - 0s 969us/step - loss: 0.1184 - accuracy: 0.8544 - precision: 0.3075 - recall: 0.0885
Epoch 119/700
185/185 [==============================] - 0s 974us/step - loss: 0.1398 - accuracy: 0.8459 - precision: 0.2773 - recall: 0.1092
Epoch 120/700
185/185 [==============================] - 0s 966us/step - loss: 0.1307 - accuracy: 0.8516 - precision: 0.3190 - recall: 0.1166
Epoch 121/700
185/185 [==============================] - 0s 976us/step - loss: 0.1150 - accuracy: 0.8590 - precision: 0.3650 - recall: 0.1017
Epoch 122/700
185/185 [==============================] - 0s 988us/step - loss: 0.1090 - accuracy: 0.8595 - precision: 0.3619 - recall: 0.0943
Epoch 123/700
185/185 [==============================] - 0s 934us/step - loss: 0.1246 - accuracy: 0.8525 - precision: 0.3039 - recall: 0.0968
Epoch 124/700
185/18

185/185 [==============================] - 0s 940us/step - loss: 0.0982 - accuracy: 0.8676 - precision: 0.4790 - recall: 0.1133
Epoch 176/700
185/185 [==============================] - 0s 956us/step - loss: 0.0902 - accuracy: 0.8728 - precision: 0.5874 - recall: 0.1001
Epoch 177/700
185/185 [==============================] - 0s 938us/step - loss: 0.0907 - accuracy: 0.8714 - precision: 0.5567 - recall: 0.0935
Epoch 178/700
185/185 [==============================] - 0s 964us/step - loss: 0.0889 - accuracy: 0.8781 - precision: 0.7073 - recall: 0.1199
Epoch 179/700
185/185 [==============================] - 0s 945us/step - loss: 0.0911 - accuracy: 0.8736 - precision: 0.5947 - recall: 0.1117
Epoch 180/700
185/185 [==============================] - 0s 949us/step - loss: 0.0924 - accuracy: 0.8725 - precision: 0.5721 - recall: 0.1084
Epoch 181/700
185/185 [==============================] - 0s 977us/step - loss: 0.0940 - accuracy: 0.8688 - precision: 0.4982 - recall: 0.1117
Epoch 182/700
185/18

185/185 [==============================] - 0s 962us/step - loss: 0.0848 - accuracy: 0.8790 - precision: 0.6598 - recall: 0.1588
Epoch 234/700
185/185 [==============================] - 0s 960us/step - loss: 0.0826 - accuracy: 0.8803 - precision: 0.7059 - recall: 0.1489
Epoch 235/700
185/185 [==============================] - 0s 971us/step - loss: 0.0826 - accuracy: 0.8811 - precision: 0.7171 - recall: 0.1530
Epoch 236/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0826 - accuracy: 0.8789 - precision: 0.6620 - recall: 0.1555
Epoch 237/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0818 - accuracy: 0.8815 - precision: 0.7231 - recall: 0.1555
Epoch 238/700
185/185 [==============================] - 0s 968us/step - loss: 0.0812 - accuracy: 0.8829 - precision: 0.7611 - recall: 0.1555
Epoch 239/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0833 - accuracy: 0.8814 - precision: 0.7046 - recall: 0.1638
Epoch 240/700
185/185 [===

...

185/185 [==============================] - 0s 997us/step - loss: 0.0473 - accuracy: 0.9279 - precision: 0.8931 - recall: 0.5112
Epoch 466/700
185/185 [==============================] - 0s 985us/step - loss: 0.0470 - accuracy: 0.9271 - precision: 0.8809 - recall: 0.5136
Epoch 467/700
185/185 [==============================] - 0s 986us/step - loss: 0.0474 - accuracy: 0.9259 - precision: 0.8663 - recall: 0.5145
Epoch 468/700
185/185 [==============================] - 0s 985us/step - loss: 0.0464 - accuracy: 0.9301 - precision: 0.8863 - recall: 0.5352
Epoch 469/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0464 - accuracy: 0.9305 - precision: 0.8923 - recall: 0.5343
Epoch 470/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0473 - accuracy: 0.9269 - precision: 0.8690 - recall: 0.5211
Epoch 471/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0486 - accuracy: 0.9289 - precision: 0.8711 - recall: 0.5368
Epoch 472/700
185/185 [===

185/185 [==============================] - 0s 1ms/step - loss: 0.0440 - accuracy: 0.9328 - precision: 0.8840 - recall: 0.5608
Epoch 524/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0406 - accuracy: 0.9390 - precision: 0.9017 - recall: 0.5997
Epoch 525/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0421 - accuracy: 0.9357 - precision: 0.8775 - recall: 0.5922
Epoch 526/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0436 - accuracy: 0.9346 - precision: 0.8750 - recall: 0.5848
Epoch 527/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0423 - accuracy: 0.9372 - precision: 0.8957 - recall: 0.5897
Epoch 528/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0425 - accuracy: 0.9343 - precision: 0.8628 - recall: 0.5931
Epoch 529/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0427 - accuracy: 0.9353 - precision: 0.8815 - recall: 0.5848
Epoch 530/700
185/185 [===========

185/185 [==============================] - 0s 1ms/step - loss: 0.0361 - accuracy: 0.9477 - precision: 0.9117 - recall: 0.6658
Epoch 582/700
185/185 [==============================] - 0s 993us/step - loss: 0.0361 - accuracy: 0.9451 - precision: 0.9054 - recall: 0.6493
Epoch 583/700
185/185 [==============================] - 0s 998us/step - loss: 0.0361 - accuracy: 0.9464 - precision: 0.9086 - recall: 0.6576
Epoch 584/700
185/185 [==============================] - 0s 981us/step - loss: 0.0381 - accuracy: 0.9418 - precision: 0.8827 - recall: 0.6410
Epoch 585/700
185/185 [==============================] - 0s 985us/step - loss: 0.0354 - accuracy: 0.9447 - precision: 0.8985 - recall: 0.6518
Epoch 586/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0371 - accuracy: 0.9436 - precision: 0.8910 - recall: 0.6493
Epoch 587/700
185/185 [==============================] - 0s 982us/step - loss: 0.0401 - accuracy: 0.9421 - precision: 0.8814 - recall: 0.6452
Epoch 588/700
185/185 [=

185/185 [==============================] - 0s 1ms/step - loss: 0.0304 - accuracy: 0.9540 - precision: 0.9225 - recall: 0.7089
Epoch 640/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0296 - accuracy: 0.9562 - precision: 0.9277 - recall: 0.7221
Epoch 641/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0325 - accuracy: 0.9493 - precision: 0.9005 - recall: 0.6890
Epoch 642/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0337 - accuracy: 0.9487 - precision: 0.9080 - recall: 0.6774
Epoch 643/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0312 - accuracy: 0.9559 - precision: 0.9239 - recall: 0.7229
Epoch 644/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0414 - accuracy: 0.9408 - precision: 0.8615 - recall: 0.6534
Epoch 645/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0309 - accuracy: 0.9551 - precision: 0.9307 - recall: 0.7105
Epoch 646/700
185/185 [===========

185/185 [==============================] - 0s 1ms/step - loss: 0.0276 - accuracy: 0.9606 - precision: 0.9334 - recall: 0.7535
Epoch 698/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0263 - accuracy: 0.9616 - precision: 0.9367 - recall: 0.7585
Epoch 699/700
185/185 [==============================] - 0s 1ms/step - loss: 0.0271 - accuracy: 0.9586 - precision: 0.9303 - recall: 0.7395
Epoch 700/700
29/29 [==============================] - 0s 724us/step - loss: 0.0371 - accuracy: 0.9459 - precision: 0.6762 - recall: 0.8161
K fold average accuracy is: 93.6894%,average precision is: 73.0692%,average recall is: 80.5535%
(9223, 11)
(9223, 1)
8641
582
973
validation accuracy is: 0.936896888214247. Successfully detected cancer: 973
precision score is: 73.7680%
recall_score is: 80.4797%
f1_score: 76.9778%


validation accuracy is: 0.9664968014745744. Successfully detected cancer: 918
precision score is: 98.0769%
recall_score is: 75.9305%
f1_score: 85.5944%

# Test Run - KNN CT-To-Cancer Prediction
In this test run we tried to use CT_Data to predict cancer, the algorithm we used here is K-nearest-Neighbours

In [5]:
if test_case == 1:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler

    param_grid = [
        {
            'weights': ['uniform'], 
            'n_neighbors': [i for i in range(1, 20)]
        },
        {
            'weights': ['distance'],
            'n_neighbors': [i for i in range(1, 20)], 
            'p': [i for i in range(1, 6)]
        }
    ]



    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    print(X.shape)
    print(y.shape)
    knn_clf = KNeighborsClassifier()
    grid_search = GridSearchCV(knn_clf, param_grid , cv = 5, scoring = 'average_precision')
    grid_search.fit(X, y)
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)


    # ============================================
    def predict_Knn(test_vec, train_data, train_label, k):
        res_list = []
        dis_list = []
        train_data = train_data.astype(float)
        test_vec = test_vec.astype(float)
        for i in range(train_data.shape[1]):
            diff = (test_vec.reshape((train_data.shape[0],1))) - train_data[:,i]
            dis_list.append(np.linalg.norm(diff))

            
        # Smallest K elements indices
        # using sorted() + lambda + list slicing
        res = sorted(range(len(dis_list)), key = lambda sub: dis_list[sub])[:k] 

        for i in range(len(res)):
            res_list.append(train_label[res[i]])

        count_0 = 0
        count_1 = 0

        for i in range(len(res_list)):
            if res_list[i] == 0:
                count_0 = count_0 + 1
            else:
                count_1 = count_1 + 1

        #if count_1 > count_0:
        #    return 1
        #else:
        #    return 0        

        # as long as one nearest neighbour has cancer,
        # than we predict cancer
        if count_1 > 0:
            return 1
        else:
            return 0
    

# Test Case 2 - Keras CT+Clinic_Data-To-Death Prediction
This is a test run perform prediction of death using (CT Data + Clinic Data). This test run uses a neural network constructed using Keras

In [11]:
if test_case == 2:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X1 = np.array(CT_Data)
    X1 = X1.T
    X1 = X1.astype(float)
    print(X1.shape)

    X2 = np.array(Clininc_Data)
    X2 = X2.T
    X2 = X2.astype(float)
    print(X2.shape)

    X = np.hstack((X1,X2))
    print(X.shape)

    y = np.array(Outcome_Data[0])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=25, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=1000, batch_size=50,verbose=1)
    # evaluate the keras model
    _, accuracy,precision,recall = model.evaluate(X, y)
    print('Accuracy: %.2f' % (accuracy*100))
    print('precision: %.2f' % (precision*100))
    print('recall rate: %.2f' % (recall*100))
    accuracy,precision,recall = kFoldCV(10, X, y, model)
    print("K fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
          .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    death_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                death_dectected = death_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(death_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected death: "+str(death_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

(9223, 11)
(9223, 14)
(9223, 25)
Epoch 1/1000
185/185 [==============================] - 1s 1ms/step - loss: 16.9907 - accuracy: 0.8860 - precision: 0.1162 - recall: 0.1384
Epoch 2/1000
185/185 [==============================] - 0s 1ms/step - loss: 2.5815 - accuracy: 0.9051 - precision: 0.2015 - recall: 0.2004 
Epoch 3/1000
185/185 [==============================] - 0s 1ms/step - loss: 2.3288 - accuracy: 0.9042 - precision: 0.2004 - recall: 0.2040
Epoch 4/1000
185/185 [==============================] - 0s 1ms/step - loss: 1.9537 - accuracy: 0.9083 - precision: 0.2275 - recall: 0.2259 
Epoch 5/1000
185/185 [==============================] - 0s 1ms/step - loss: 1.6386 - accuracy: 0.9071 - precision: 0.2116 - recall: 0.2058 
Epoch 6/1000
185/185 [==============================] - 0s 1ms/step - loss: 1.6991 - accuracy: 0.9092 - precision: 0.2353 - recall: 0.2332 
Epoch 7/1000
185/185 [==============================] - 0s 1ms/step - loss: 1.5554 - accuracy: 0.9075 - precision: 0.2236 - reca

185/185 [==============================] - 0s 1ms/step - loss: 0.5659 - accuracy: 0.9237 - precision: 0.3619 - recall: 0.3698 
Epoch 60/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.7175 - accuracy: 0.9175 - precision: 0.3059 - recall: 0.3042 
Epoch 61/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.7401 - accuracy: 0.9184 - precision: 0.3172 - recall: 0.3224 
Epoch 62/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.4690 - accuracy: 0.9246 - precision: 0.3710 - recall: 0.3825 
Epoch 63/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.5672 - accuracy: 0.9230 - precision: 0.3555 - recall: 0.3607
Epoch 64/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.5471 - accuracy: 0.9277 - precision: 0.3899 - recall: 0.3807
Epoch 65/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.4632 - accuracy: 0.9258 - precision: 0.3797 - recall: 0.3880
Epoch 66/1000
185/185 [=======

185/185 [==============================] - 0s 1ms/step - loss: 0.2924 - accuracy: 0.9329 - precision: 0.4359 - recall: 0.4335
Epoch 118/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.2999 - accuracy: 0.9287 - precision: 0.4018 - recall: 0.4062
Epoch 119/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.2533 - accuracy: 0.9321 - precision: 0.4340 - recall: 0.4608
Epoch 120/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.3754 - accuracy: 0.9278 - precision: 0.3938 - recall: 0.3953
Epoch 121/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.3767 - accuracy: 0.9279 - precision: 0.3982 - recall: 0.4135 
Epoch 122/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.2491 - accuracy: 0.9333 - precision: 0.4417 - recall: 0.4554
Epoch 123/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.2514 - accuracy: 0.9332 - precision: 0.4403 - recall: 0.4499
Epoch 124/1000
185/185 [===

185/185 [==============================] - 0s 1ms/step - loss: 0.1342 - accuracy: 0.9410 - precision: 0.5044 - recall: 0.5173
Epoch 176/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1464 - accuracy: 0.9435 - precision: 0.5246 - recall: 0.5428
Epoch 177/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1366 - accuracy: 0.9400 - precision: 0.4965 - recall: 0.5100
Epoch 178/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.2041 - accuracy: 0.9314 - precision: 0.4261 - recall: 0.4408
Epoch 179/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1155 - accuracy: 0.9437 - precision: 0.5266 - recall: 0.5410 
Epoch 180/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1430 - accuracy: 0.9385 - precision: 0.4842 - recall: 0.5009    
Epoch 181/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1991 - accuracy: 0.9330 - precision: 0.4389 - recall: 0.4517
Epoch 182/1000
185/185 

185/185 [==============================] - 0s 1ms/step - loss: 0.0671 - accuracy: 0.9500 - precision: 0.5783 - recall: 0.5920 
Epoch 234/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0792 - accuracy: 0.9457 - precision: 0.5438 - recall: 0.5428 
Epoch 235/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0770 - accuracy: 0.9489 - precision: 0.5720 - recall: 0.5647 
Epoch 236/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0896 - accuracy: 0.9462 - precision: 0.5474 - recall: 0.5574
Epoch 237/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0995 - accuracy: 0.9413 - precision: 0.5071 - recall: 0.5191 
Epoch 238/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0988 - accuracy: 0.9459 - precision: 0.5458 - recall: 0.5428
Epoch 239/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.1213 - accuracy: 0.9403 - precision: 0.4982 - recall: 0.4973
Epoch 240/1000
185/185 [

...

185/185 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 0.9938 - precision: 0.9590 - recall: 0.9362
Epoch 756/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 0.9943 - precision: 0.9733 - recall: 0.9290
Epoch 757/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 0.9963 - precision: 0.9813 - recall: 0.9563
Epoch 758/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0076 - accuracy: 0.9913 - precision: 0.9416 - recall: 0.9107
Epoch 759/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0107 - accuracy: 0.9880 - precision: 0.9132 - recall: 0.8816
Epoch 760/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0284 - accuracy: 0.9728 - precision: 0.7759 - recall: 0.7632
Epoch 761/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0138 - accuracy: 0.9846 - precision: 0.8847 - recall: 0.8525
Epoch 762/1000
185/185 [====

185/185 [==============================] - 0s 1ms/step - loss: 0.0181 - accuracy: 0.9794 - precision: 0.8282 - recall: 0.8251 
Epoch 814/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 0.9896 - precision: 0.9202 - recall: 0.9035
Epoch 815/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0092 - accuracy: 0.9904 - precision: 0.9323 - recall: 0.9035
Epoch 816/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 0.9923 - precision: 0.9442 - recall: 0.9253
Epoch 817/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0064 - accuracy: 0.9922 - precision: 0.9392 - recall: 0.9290
Epoch 818/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0368 - accuracy: 0.9721 - precision: 0.7694 - recall: 0.7596
Epoch 819/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0213 - accuracy: 0.9810 - precision: 0.8502 - recall: 0.8270
Epoch 820/1000
185/185 [===

185/185 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 0.9976 - precision: 0.9835 - recall: 0.9763
Epoch 872/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0140 - accuracy: 0.9847 - precision: 0.8893 - recall: 0.8488
Epoch 873/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.9825 - precision: 0.8579 - recall: 0.8470
Epoch 874/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0063 - accuracy: 0.9914 - precision: 0.9417 - recall: 0.9126
Epoch 875/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 0.9962 - precision: 0.9759 - recall: 0.9599
Epoch 876/1000
185/185 [==============================] - 0s 997us/step - loss: 0.0029 - accuracy: 0.9965 - precision: 0.9832 - recall: 0.9581
Epoch 877/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0161 - accuracy: 0.9849 - precision: 0.8868 - recall: 0.8561
Epoch 878/1000
185/185 [==

185/185 [==============================] - 0s 1000us/step - loss: 0.0011 - accuracy: 0.9992 - precision: 0.9963 - recall: 0.9909
Epoch 929/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 0.9975 - precision: 0.9852 - recall: 0.9727
Epoch 930/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9990 - precision: 0.9945 - recall: 0.9891
Epoch 931/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 0.9940 - precision: 0.9540 - recall: 0.9454
Epoch 932/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 0.9873 - precision: 0.8985 - recall: 0.8871
Epoch 933/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0363 - accuracy: 0.9708 - precision: 0.7555 - recall: 0.7541
Epoch 934/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0117 - accuracy: 0.9864 - precision: 0.9030 - recall: 0.8652
Epoch 935/1000
185/185 [=

185/185 [==============================] - 0s 1ms/step - loss: 0.0310 - accuracy: 0.9761 - precision: 0.8133 - recall: 0.7778
Epoch 986/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0141 - accuracy: 0.9859 - precision: 0.8816 - recall: 0.8816
Epoch 987/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 0.9974 - precision: 0.9870 - recall: 0.9690
Epoch 988/1000
185/185 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 0.9978 - precision: 0.9871 - recall: 0.9763
Epoch 989/1000
185/185 [==============================] - 0s 988us/step - loss: 0.0010 - accuracy: 0.9995 - precision: 0.9982 - recall: 0.9927
Epoch 990/1000
185/185 [==============================] - 0s 1ms/step - loss: 7.6302e-04 - accuracy: 0.9999 - precision: 1.0000 - recall: 0.9982
Epoch 991/1000
185/185 [==============================] - 0s 995us/step - loss: 7.1348e-04 - accuracy: 0.9997 - precision: 1.0000 - recall: 0.9945
Epoch 992/1000
1

validation accuracy is: 0.9998915754093028. Successfully detected death: 548
precision score is: 100.0000%
recall_score is: 99.8179%
f1_score: 99.9088%

# Test Case 3 - Keras CT-To-Death Prediction
This is a test run perform prediction of death using (CT Data Only). This test run uses a neural network constructed using Keras

In [7]:
if test_case == 3:
    import numpy as np
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.preprocessing import StandardScaler 

    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)


    y = np.array(Outcome_Data[0])
    y = y.astype(int)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)

    # define the keras model
    model = Sequential()
    model.add(Dense(32, input_dim=25, use_bias=True))
    model.add(Dense(64, activation='relu',use_bias=True))
    model.add(Dense(128, activation='relu',use_bias=True))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    # compile the keras model
    model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=1000, batch_size=50,verbose=1)
    # evaluate the keras model
    _, accuracy,precision,recall = model.evaluate(X, y)
    print('Accuracy: %.2f' % (accuracy*100))
    print('precision: %.2f' % (precision*100))
    print('recall rate: %.2f' % (recall*100))
    


    # =========================================================
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    test_y = model.predict(X)
    print(X.shape)
    print(test_y.shape)
    test_y = test_y > 0.5
    right_count = 0
    wrong_count = 0
    death_dectected = 0
    for i in range(len(test_y)):
        if test_y[i] == y[i]:
            right_count = right_count + 1
            if test_y[i] == True:
                death_dectected = death_dectected + 1
        else:
            wrong_count = wrong_count + 1
    print(right_count)
    print(wrong_count)
    print(death_dectected)
    print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected death: "+str(death_dectected))
    print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
    print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
    print("f1_score: {0:.4%}".format(f1_score(y, test_y)))